#### Vscode has a Dev Containers extension for creating Dockerized development environments

https://marketplace.visualstudio.com/items?itemName=ms-vscode-remote.remote-containers

This guide includes the instructions for creating a Dev Container environment for running the Shiny App using R 3.6.3.

#### Start by building the sec_poc/R image

In [ ]:
!docker build -t sec_poc/R .

#### Start the docker application with docker-compose

In [ ]:
# start the application in detached mode
!docker-compose up -d

Now inside of Vscode (making sure that the dev container extension is installed), you should be able to connect to the R App container.

See the command > Dev Containers: Attach to Running Container...

#### Install these system packages for better developer experience

In [ ]:
!apt update
# For tracking file changes. If mounting as a Docker volume, the .git/ folder should already be initialized to the sec_poc repo.
!apt install git
# For watching file changes while running shiny::runApp. See sec_poc/watch.sh
!apt install inotify-tools
# For Python ctypes, issues occurred when using pyenv to install 3.10
!apt install libffi-dev
# Installs psql to connect to the Postgres container
!apt install postgresql-client


#### R Debugger Extension
https://marketplace.visualstudio.com/items?itemName=RDebugger.r-debugger

The R Debugger extension relies on the vscDebugger R package. To install it to R 3.6, you must build it from source.

1. Copy the source code from https://github.com/ManuelHentschel/vscDebugger. 

Copy it from the host machine and paste it in the devContainer's main workspace directory (which should be the shared volume).

2. Build it,

In [ ]:
!Rscript vscDebugger/build.R

The build will complain about building vignettes which relies on Pandoc.


In [ ]:
!R -e "install.packages('pandoc')"

In [ ]:
# Install the pandoc binary
!R -e "pandoc::pandoc_install()"

This will likely install it to `/root/.local/share` which you should symlink to `/usr/local/bin`.

In [ ]:
!ln -s /root/.local/share/r-pandoc/3.1.12.3/pandoc-3.1.12.3/bin/pandoc /usr/local/bin/

In [ ]:
# Verify that pandoc is in the PATH
!which pandoc

The build.R script does not make the folders it creates writable, so it will encounter Permissions denied errors on file copies to these created folders. As this happens, delete the folder with insufficient write permissions and recreate it as the root user (write should be enabled by default).

In [ ]:
# For example,
#!rm -rf vscDebugger/docs
#!mkdir vscDebugger/docs

Once the build succeeds, the vscDebugger package should be installed to the site-library LibPath

#### R Language Extension
https://marketplace.visualstudio.com/items?itemName=REditorSupport.r

In [ ]:
# Install the required extension packages
!R -e "install.packages('languageserver')"

:tada: The vscode dev container should be ready to go! You now have R syntax highlighting, intellisense, and debugging built-in to Vscode.

#### Additional software

- Pyenv

Python is used for database ETL scripts.

In [ ]:
!curl https://pyenv.run | bash
# Add the init commands to ~/.bashrc

- Starship (Customizable shell prompt) (optional)

In [ ]:
!curl -sS https://starship.rs/install.sh | sh
# Add the init command to ~/.bashrc

#### Docker networking

Assuming that the R App and PostgreSQL containers are in the same network, the container name can be used as the hostname when connecting to the container. Or the hostname can be `host.docker.internal`. `host.docker.internal` can also be used to connect from the host to the container on an exposed port. For example, `psql -h host.docker.internal -d sec -U secapp`

#### PostgreSQL

It's easier to save the connection params in ~/.pgpass so that psql doesn't prompt for password everytime

#### Jupyter

See this guide for using R in Jupyter Notebooks. Make sure that Jupyter is installed in the Python environment.
https://stackoverflow.com/questions/56497422/using-jupyter-r-kernel-with-visual-studio-code